# Importing AFM data into HyperSpy for plotting

In [1]:
%matplotlib qt5

In [2]:
import numpy as np
import hyperspy.api as hs

## Datafiles

While Gwyddion is nice for processing the AFM data, it is nice to do the plotting using matplotlib. This notebook shows how to import `.xyz` files into a HyperSpy signal.

Firstly:
- Do the processing you want to do using Gwyddion
- Then, select the data you want to save, by clicking the window with that image. In Gwyddion, go to `File`, `Save As`
- Next, in the lower left in the "Save File" window, click "File type". Select "XYZ text data (.xyz)"
- In the top part of the "Save File" window, change the "Name" to what you want to call the file + .xyz extension
- Click "Save" (bottom right), this will open a new window ("Export XYZ").
- Toggle "Add informational comment header", then click "OK" to save the file.

Run the large cell below, this defines a python function for parsing the AFM data into a NumPy array, which is then used to make a HyperSpy signal.

In [ ]:
def import_xyz_data(filename):
    with open(filename) as f:
        title = f.readline()[11:]
        title = title.replace("\n", "")

    xyz_data = np.loadtxt(filename, usecols=(0, 1, 2))

    x_list = xyz_data[:, 0]
    y_list = xyz_data[:, 1]
    z_list = xyz_data[:, 2]

    # To micrometer in xy-dimensions
    x_list *= 1e6
    y_list *= 1e6

    # To nanometer in z-dimension
    z_list *= 1e9

    x_set_list = list(set(x_list))
    y_set_list = list(set(y_list))
    x_set_list.sort()
    y_set_list.sort()

    x_size = len(x_set_list)
    y_size = len(y_set_list)

    x_scale = (x_set_list[-1] - x_set_list[0]) / x_size
    y_scale = (y_set_list[-1] - y_set_list[0]) / y_size

    x_index_list = np.zeros_like(x_list).astype(np.uint16)
    y_index_list = np.zeros_like(y_list).astype(np.uint16)

    for nx, x in enumerate(x_list):
        for ix, x_set in enumerate(x_set_list):
            if x == x_set:
                x_index_list[nx] = ix

    for ny, y in enumerate(y_list):
        for iy, y_set in enumerate(y_set_list):
            if y == y_set:
                y_index_list[ny] = iy

    data_array = np.zeros((y_size, x_size), dtype=np.float32)
    for ii in range(len(x_index_list)):
        x_index = x_index_list[ii]
        y_index = y_index_list[ii]
        data_array[y_index, x_index] = z_list[ii]

    s = hs.signals.Signal2D(data_array)
    s.axes_manager[0].scale = x_scale
    s.axes_manager[1].scale = x_scale
    s.axes_manager[0].units = "um"
    s.axes_manager[1].units = "um"
    s.axes_manager[0].name = "Probe x"
    s.axes_manager[1].name = "Probe y"
    s.metadata.General.title = title

    return s

Then, run this function with the .xyz file as a parameter

In [ ]:
s = import_xyz_data("test.xyz")

In [ ]:
s.plot()

## Getting line profiles

In [ ]:
def get_line_profile(s):
    roi = hs.roi.Line2DROI(linewidth=0.5)
    s.plot()
    s_roi = roi.interactive(s)
    s_roi.plot(autoscale="xyv")
    roi_span = hs.roi.SpanROI()
    roi_span.interactive(s_roi)
    return s_roi

In [ ]:
s_line_profile = get_line_profile(s)

In [ ]:
s_line_profile.plot()

### Plotting the data

Now, you can use the matplotlib library to visualize the results, just as the other computer exercises.